# Spark Recommender System

In [27]:
# import necessary modules
import os
import shutil
import pyspark as ps
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.sql import Row
from pyspark.sql.types import DoubleType

In [28]:
# create spark context
spark = (ps.sql.SparkSession.builder
        .appName("ALS model")
        .getOrCreate()
        )
sc = spark.sparkContext
print(spark.version)

3.5.0


## Read in Data

We will use user clusters as part of user features for our model here. 

In [29]:
# source data from prior step
data_dir = os.path.join("clean_data", "")
file = os.path.join(data_dir, "spark_master.csv")

# options are specified to read in data without error
df_master = spark.read.format("csv")\
               .option("multiline", "true")\
               .option("header", "true")\
               .option("inferSchema", "true")\
               .load(file)

In [ ]:
user_features = df_master.select(df_master['customer_unique_id'], 
                               df_master['product_id'], 
                               df_master['review_score'])
user_features = user_features.dropna()

In [ ]:
item_features = df_master.select(df_master['product_id'], 
                               df_master['product_category_name'])
item_features = item_features.dropna()

In [ ]:
from pyspark.ml.feature import StringIndexer

# create object of StringIndexer class and specify input and output column
SI_customer = StringIndexer(inputCol='customer_unique_id',outputCol='customer_index')
SI_product = StringIndexer(inputCol='product_id',outputCol='product_index')

# transform the data
user_features = SI_customer.fit(user_features).transform(user_features)
user_features = SI_product.fit(user_features).transform(user_features)

# view the transformed data
user_features.select('customer_unique_id', 'customer_index', 'product_id', 'product_index').show(10)

+--------------------+--------------+--------------------+-------------+
|  customer_unique_id|customer_index|          product_id|product_index|
+--------------------+--------------+--------------------+-------------+
|0000366f3b9a7992b...|        5166.0|372645c7439f9661f...|        451.0|
|0000b849f77a49e4a...|        5167.0|5099f7000472b634f...|       2313.0|
|0000f46a3911fa3c0...|        5168.0|64b488de448a5324c...|       3486.0|
|0000f6ccb0745a6a4...|        5169.0|2345a354a6f203360...|       5592.0|
|0004aac84e0df4da2...|        5170.0|c72e18b3fe2739b8d...|      27381.0|
|0004bd2a26a76fe21...|        5171.0|25cf184645f3fae66...|       5607.0|
|00050ab1314c0e55a...|        5172.0|8cefe1c6f2304e7e6...|       2900.0|
|00053a61a98854899...|         726.0|62984ea1bba7fcea1...|       4526.0|
|00053a61a98854899...|         726.0|58727e154e8e85d84...|       1134.0|
|0005e1862207bf6cc...|        5173.0|e24f73b7631ee3fbb...|        887.0|
+--------------------+--------------+--------------

In [63]:
# generate top_n product recommendations for user
nrecommend = 5
user_recs = final_model.recommendForAllUsers(nrecommend)
user_recs.show(4)

+--------------+--------------------+
|customer_index|     recommendations|
+--------------+--------------------+
|            28|[{6490, 1.3932567...|
|            31|[{2857, 4.959978}...|
|            34|[{879, 4.951983},...|
|            53|[{11275, 4.985459...|
+--------------+--------------------+
only showing top 4 rows



In [64]:
recs = user_recs.toPandas()

## Recommender Function

In [68]:
# Generate pandas df for accessing products in recommender function
products = item_features.toPandas()

KeyboardInterrupt: 

In [ ]:
user_features_df = user_features.toPandas()

In [ ]:
def user_recommendations(user_id, top_n = 3):
    
    if top_n > nrecommend:
        print("Please select up to {} items to recommend".format(nrecommend))
        return; 
    
    prior_purchases = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                                                        ['product_id'].unique()
    num_items = len(prior_purchases)
    
    if num_items < 3:
        items = num_items
    else:
        items = 3
    
    print("User: {}\n".format(user_id))
    print("Known positives: ")
    for n in range(items):
        known_like_product = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                            ['product_id'].unique()[n]
        known_like_category = products[products['product_id'] == known_like_product]\
                                                            ['product_category_name'].unique()[0]
    
        print("\t", known_like_product)
        print("\t", known_like_category, "\n")
    
    
    customer_index = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                            ['customer_index'].unique()[0]
    print("Top {} Recommendations: \n".format(top_n))
    rec_products = []
    
    for n in range(top_n):
        
        rec_products.append(list(recs[recs['customer_index'] == customer_index]['recommendations'])[0][n][0])
        
        print("{}.\n".format(n+1), products[products['product_index'] == rec_products[n]]\
                                                  [['product_id', 'product_category_name']].iloc[0][0])
        
        print(products[products['product_index'] == rec_products[n]]\
                                                  [['product_id', 'product_category_name']].iloc[0][1])

__Test for customer_id = 'c8ed31310fc440a3f8031b177f9842c3'__

In [ ]:
user_recommendations('c8ed31310fc440a3f8031b177f9842c3', top_n=5)

User: c8ed31310fc440a3f8031b177f9842c3

Known positives: 
	 1065e0ebef073787a7bf691924c60eeb
	 construction_tools_construction 

	 0cf2faf9749f53924cea652a09d8e327
	 construction_tools_construction 

	 309dd69eb83cea38c51709d62befe1a4
	 construction_tools_construction 

Top 5 Recommendations: 

1.
 15b1f9b06d0e709552d7d8638387e09b
furniture_decor
2.
 7189fb70393a0b87189f93f19655f8db
toys
3.
 3e7ec3672e5549ba74cf635752bfc70b
furniture_decor
4.
 14ad6805c263d8d758d648f46a06570e
baby
5.
 329c661807f085964b1877bfeca6ff73
furniture_decor


__Test for customer_id = '698e1cf81d01a3d389d96145f7fa6df8'__

In [ ]:
user_recommendations('698e1cf81d01a3d389d96145f7fa6df8', top_n=5)

User: 698e1cf81d01a3d389d96145f7fa6df8

Known positives: 
	 9571759451b1d780ee7c15012ea109d4
	 auto 

Top 5 Recommendations: 

1.
 0a4f9f421af66d2ea061fbb8883419f7
health_beauty
2.
 fdd84aefb08c8f8225e0b8c97429d53b
health_beauty
3.
 12485f9cdebb6ca179826ede539554ad
air_conditioning
4.
 616042729c11849827291496b18e9ec5
sports_leisure
5.
 7a5c07212703b5f01ee199d29a29a587
cool_stuff


__Test for customer_id = '89be58cbdd6ef318e3ed93fdb22be178'__

In [ ]:
user_recommendations('89be58cbdd6ef318e3ed93fdb22be178', top_n=5)

User: 89be58cbdd6ef318e3ed93fdb22be178

Known positives: 
	 3fdb534dccf5bc9ab0406944b913787d
	 diapers_and_hygiene 

Top 5 Recommendations: 

1.
 779dd392d4fbe5ca656bf3ceabecbf0b
construction_tools_construction
2.
 bdcf6a834e8faa30dac3886c7a58e92e
health_beauty
3.
 91b08d34d0ba4db44da2dc382867ba49
telephony
4.
 1b8ee158f59c098470fad33f39660964
furniture_living_room
5.
 d9339c5714743c460a9470730f79f6c5
computers_accessories


## Summary of Results

It's clear after producing a simple recommendation system with matrix factorization using only users prior purchase history that this dataset simply does not have the data necessary to give accurate results. Many of the attempted recommendations produced results that are clearly not relevant for the user. As can be seen in the suggestions above, many of the top items are from categories very different from the original purchase. 

Comparing these same recommendations with LightFM shows how well hybrid recommenders can do for data sets like this one, with very few return users. 